In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
directory_train = 'F:\\Projects\\data\\Trainset\\'
directory_valid = 'F:\\Projects\\data\\Validset\\'
directory_test = 'F:\\Projects\\data\\Testset\\'

In [3]:
df_train = pd.read_csv('F:\\Projects\\data\\trainset.csv')
df_valid = pd.read_csv('F:\\Projects\\data\\validset.csv')
df_test = pd.read_csv('F:\\Projects\\data\\testset.csv')

In [4]:
df_train

,img_name,age
0,100_0_0_20170112213500903.jpg.chip.jpg,100
1,100_0_0_20170112215240346.jpg.chip.jpg,100
2,100_1_0_20170110183726390.jpg.chip.jpg,100
3,100_1_0_20170112213001988.jpg.chip.jpg,100
4,100_1_0_20170112213303693.jpg.chip.jpg,100
...,...,...
17034,9_1_2_20170104020210475.jpg.chip.jpg,9
17035,9_1_3_20161220222856346.jpg.chip.jpg,9
17036,9_1_3_20170104222949455.jpg.chip.jpg,9
17037,9_1_4_20170103200637399.jpg.chip.jpg,9


In [5]:
train_file_paths = directory_train + df_train['img_name'].values
valid_file_paths = directory_valid + df_valid['img_name'].values

In [6]:
train_labels = df_train['age'].values
valid_labels = df_valid['age'].values

In [7]:
train_labels

array([100, 100, 100, ...,   9,   9,   9], dtype=int64)

In [8]:
ds_train = tf.data.Dataset.from_tensor_slices((train_file_paths, train_labels))
ds_valid = tf.data.Dataset.from_tensor_slices((valid_file_paths, valid_labels))

In [9]:
def read_image(image_file, label):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_image(image, channels=3, dtype=tf.float32)
    return image, label

In [10]:
def augment(image, label):
    # data augmentation here
    # image = tf.image.random_brightness(x, max_delta=0.05)
    return image, label

In [11]:
ds_train = ds_train.map(read_image).batch(64)
ds_valid = ds_valid.map(read_image).batch(64)

### Model

In [12]:
from tensorflow.keras import datasets, layers, models

In [13]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 173056)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                1

In [15]:
# Compile the model
learning_rate = 0.01

model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate), 
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mse']
)

In [16]:
model.fit(ds_train, epochs=5, validation_data=ds_valid)

Epoch 1/5


ValueError: in user code:

    c:\users\lubos\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    c:\users\lubos\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\lubos\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\lubos\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\lubos\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\users\lubos\anaconda3\envs\tf_gpu\lib\sit